Kalibroinnit jokainen omaan json fileen:
- Name
- Description
- Time-stamp last edit time
- Weight
- Data

Tätä varten tarvitaan load, save, select ja delete funktio
Select ja delete listaa telineet timestampilla sortattuna ja näyttää Time stampin ja nimen. Valitun telineen kuvaaja ja description näytetään interaktiivisella widgetillä. Created time timestamp voi olla myös nimessä niin tyhjäkin nimi on ok.

In [1]:
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
import glob
import os

In [2]:
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
cal = pd.read_pickle('specimenHolder')
cal.head()

,holder1
0.0,-11.930027
0.5,-16.603760
1.0,-15.658180
1.5,-10.848916
2.0,-15.057949


# Save

In [4]:
name = 'name'
description = 'Description text goes here.'
weight = 8453.453

outdata = {}
timestamp = time.time()
outdata['name'] = '{}_{}'.format(name, datetime.fromtimestamp(timestamp).isoformat())
outdata['edit_time'] = timestamp
outdata['description'] = description
outdata['weight'] = weight
outdata['cal'] = cal.to_json()

In [5]:
with open('holders/{}_{}.json'.format(name, datetime.fromtimestamp(timestamp).isoformat()), 'w') as outfile:
    json.dump(outdata, outfile)

# Load

In [17]:
def loadHolders():
    holders = []
    holderFiles = glob.glob('holders/*.json')
    
    print(holderFiles)
    
    for holderFile in holderFiles:
        with open(holderFile) as json_file:
            holder = json.load(json_file)
            holders.append(holder)

    firstHolder = True
    for holder in holders:
        calData = pd.read_json(holder['cal'], convert_axes=False)
        calData.index = calData.index.astype(float)
        calData[calData.columns] = calData[calData.columns].astype(float)
        calData = calData.sort_index()
        holder['cal'] = calData

        if(firstHolder):
            firstHolder = False
            holdersDF = pd.DataFrame(holder['cal'])
            holdersDF.rename(columns={holdersDF.columns[0] : holder['name']}, inplace = True)
        else:
            holdersDF[holder['name']] = holder['cal']
        
    return holders, holdersDF


In [18]:
holders, holdersDF = loadHolders()

['holders/name_2019-11-04T16:38:57.424299.json', 'holders/name_2019-11-04T16:12:22.724541.json', 'holders/name_2019-11-04T16:38:55.305776.json', 'holders/name_2019-11-05T11:18:03.390417.json', 'holders/Holder2_2019-11-05T11:03:46-Copy1.867824.json']


In [19]:
holdersDF.head()

,name_2019-11-04T16:38:57.424299,name_2019-11-04T16:12:22.724541,name_2019-11-04T16:38:55.305776,name_2019-11-05T11:18:03.390417,Holder2_2019-11-05T11:03:46.867824
0.0,-11.930027,-11.930027,-11.930027,-11.930027,7.867046
0.5,-16.603760,-16.603760,-16.603760,-16.603760,-15.519568
1.0,-15.658180,-15.658180,-15.658180,-15.658180,-15.868679
1.5,-10.848916,-10.848916,-10.848916,-10.848916,6.291104
2.0,-15.057949,-15.057949,-15.057949,-15.057949,-12.303169


In [9]:
global selectedHolder
global holderSelector
global holders

def selectSpecimenHolder(df):
    global holderSelector
    global holders
    
    holderSelector = widgets.Select(
    options=df.columns,
    value=df.columns[0],
    description='Holder: ',
    disabled=False,
    layout = widgets.Layout(height='100px')
    )
    
    selButton = widgets.Button(
    description='Select',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Apply the specimen holder selection',
    icon='check'
    )
    
    delButton = widgets.Button(
    description='Delete',
    disabled=False,
    button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Delete the selected holder',
    #icon='check'
    )
    
    buttons = widgets.HBox([selButton, delButton])
    widgets.interact(plotSpecimenHolder, df=widgets.fixed(df), column=holderSelector)
    display(buttons)
    selButton.on_click(applyHolderSelection)
    delButton.on_click(deleteHolder)
    
def plotSpecimenHolder(df, column):
    global selectedHolder
    %matplotlib notebook
    fig=plt.figure(column, figsize=(4,3), dpi=80)
    plt.plot(df[column])
    plt.show()
    selectedHolder = column

def applyHolderSelection(b):
    clear_output()
    print('Selected holder: {}'.format(selectedHolder))
    
def deleteHolder(b):
    global holders
    holderFiles = glob.glob('holders/*.json')
    for file in holderFiles:
        if selectedHolder in file:
            os.remove(file)
    holders, holdersDF = loadHolders()
    holderSelector.options = holdersDF
    

In [22]:
selectSpecimenHolder(holdersDF)

interactive(children=(Select(description='Holder: ', layout=Layout(height='100px'), options=('name_2019-11-04T…